# Intro to Data Science 
## Part VII. - Regression and Embedding pipelines

### Table of contents

- #### Regression
    - <a href="#What-is-Regression?">Theory</a>
    - <a href="#Linear-regression">Linear regression</a>
    - <a href="#Ridge-regression">Ridge regression</a>
    - <a href="#Bayesian-Ridge-regression">Bayesian regression</a>
    - <a href="#Support-Vector-regression">Support Vector regression</a>
    - <a href="#XGBoost">XGBoost</a>

- #### Embedding pipelines
    - <a href="#Embedding-pipelines">Reusing trained pipelines</a>
        - <a href="#Exporting-pipelines">Exporting pipelines</a>
        - <a href="#Loading-pipelines">Loading pipelines</a>

---

## What is Regression?
Regression - just as classification - is a supervised machine learning problem however in case of regression the target variable is continuous. It is also _"a statistical process for estimating the relationships among variables. It includes many techniques for modeling and analyzing several variables, when the focus is on the relationship between a __dependent variable__ and one or more __independent variable__s (or 'predictors')."_ from: <a href="https://en.wikipedia.org/wiki/Regression_analysis">Wiki</a>

It is important to note that instead of the descriptive nature of statistical regression analysis Data Science focuses on the predictive side of this method.

## Why is it important?
_"Regression analysis is widely used for prediction and forecasting, where its use has substantial overlap with the field of machine learning."_ from: <a href="https://en.wikipedia.org/wiki/Regression_analysis">Wiki</a>

It is used to forecast any continuous variable:
- stock market
- salary prediction
- network traffic
- traffic
- etc.

## Tools
- Linear regression
- Ridge regression
- LASSO
- Bayesian regression
- Support Vector regression
- etc.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.datasets import load_boston

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

In [ ]:
def plot_pred(y, predicted):
    fig, ax = plt.subplots()
    ax.scatter(y, predicted, edgecolors='k')
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()

    
def plot_boston(ax):
    ax.scatter(lsop_train, y_train, edgecolors='k', s=10)
    ax.set_xlabel("% lower status of the population")
    ax.set_ylabel("Median value of owner-occupied homes in $1000's")
    ax.set_xlim([-10,50])
    ax.set_ylim([0,60])
    
    
def plot_curve(estimator, param, values, ax):   
    for color, value in zip(colors, values):
        estimator = estimator.set_params(**{param: value}).fit(lsop_train, y_train)
        ax.plot(curve_x, estimator.predict(curve_x), '-', c=color, lw=2, label=value)
    plot_boston(ax)
    ax.legend(loc='upper right')

    
def show_score(model, X, y, cv=10, metric=None):
    scores = cross_val_score(model, X, y, cv=cv, scoring=metric)
    return "Accuracy: {:.2f} (+/- {:.2f})".format(scores.mean(), scores.std() * 2)

colors = ['g', 'r', 'y', 'c', 'm']

In [ ]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

lsop = X[:,12][:, np.newaxis]
lsop_train = X_train[:,12][:, np.newaxis]
lsop_test = X_test[:,12][:, np.newaxis]

curve_x = np.linspace(-10,50, num=300)[..., np.newaxis]

$\newcommand{\bs}[1]{\boldsymbol{#1}}$

## Variations on a Theme

The traditional linear problem is stated like this:
$$ y_i = \bs{x}_i \bs{\beta} $$
for every observation $i$, or more compactly
$$ \bs{y} = \bs{X}\bs{\beta} $$
where $ \bs{X} $ is the matrix observed values, $\bs{y}$ is the vector of observed output variables, and $\bs{\beta}$ is the weight vector which we want to find. 

In OLS, we try to find the $\bs{\beta}$ while minimizing a *loss function*, which is simply the sum of squares of the differences between the predicted and observed values (also called sum of squared residuals or SSR), 

$ \mathrm{Cost}(\bs{\beta}) = \mathrm{SSR}(\bs{\beta}) = \sum _i (\hat y_i - y_i)^{2} $.  

<a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html">Ridge</a>, <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html">LASSO</a> and <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html">Bayesian</a> regressions (and a couple more) are basically simple <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html">linear</a> regressions, but with the loss function being modified.  
Ridge regression adds the sum of the squares of the weights with a constant multiplier to the loss, i.e.

$ \mathrm{Cost}(\bs{\beta}) = \sum _i (\hat y_i - y_i)^{2} + \alpha \sum _i \beta _i^{2}. $

LASSO adds the sum of the absolute values of the coefficients, i.e.

$ \mathrm{Cost}(\bs{\beta}) = \sum _i (\hat y_i - y_i)^{2} + \alpha \sum _i \vert \beta _i. \vert $

### Ok, but what is the point of this?

This technique is called <a href="https://en.wikipedia.org/wiki/Regularization_(mathematics)">**regularization**</a>, and the use of this in our case is to prevent the model from **overfitting** to the data (which is our greatest enemy, right before **the curse of dimensionality**). Basically it prevents the coefficients from growing too large. To illustrate this, we will use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html">*boston dataset*</a>. (You should also check out <a href="https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/">this</a> for a more detailed discussion on Ridge and LASSO)

---

## Linear regression - OLS

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
ols = Pipeline([('poly', PolynomialFeatures()), ('ols', LinearRegression())])
parameters = {'poly__degree': range(1,16)}
ols_grid = GridSearchCV(ols, parameters, n_jobs=2, scoring='neg_mean_squared_error')
ols_grid.fit(lsop_train, y_train)

In [ ]:
y_hat = ols_grid.best_estimator_.predict(lsop_test)
plot_pred(y_test, y_hat)

Plot some example curve with different degrees.

In [ ]:
fig, ax = plt.subplots()
plot_curve(ols, 'poly__degree', [1, 2, 3, 5, 13], ax)

## Ridge regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Pipeline([('poly', PolynomialFeatures(degree=5)), ('ridge', Ridge())])
params = {'ridge__alpha': np.logspace(-15, 13, 29)}
ridge_grid = GridSearchCV(ridge, params, n_jobs=2, scoring='neg_mean_squared_error')
ridge_grid.fit(lsop_train, y_train)

In [ ]:
y_hat = ridge_grid.best_estimator_.predict(lsop_test)
plot_pred(y_test, y_hat)

Plot some example curves to see how the regularization parameters "deform" the 5 degree polynomial we saw in the previous plot.

In [ ]:
fig, ax = plt.subplots()
plot_curve(ridge, 'ridge__alpha', [1e-13, 1e-1, 1e1, 1e2, 1e10], ax)

## LASSO

Least absolute shrinkage and selection operator

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Pipeline([('poly', PolynomialFeatures(degree=5)), ('lasso', Lasso(max_iter=10000))])
params = {'lasso__alpha': np.logspace(-5, 13, 19)}
lasso_grid = GridSearchCV(lasso, params, scoring='neg_mean_squared_error')
lasso_grid.fit(lsop_train, y_train)

In [ ]:
y_hat = lasso_grid.best_estimator_.predict(lsop_test)
plot_pred(y_test, y_hat)

LASSO also works as a feature selection tool, we can see that by setting the alpha high enough, it sets some coefficients to zero. Also, we can see that if we go overboard with this, it can lead to **underfitting**, which is also bad.

In [ ]:
coefs = pd.DataFrame()
pipe = Pipeline([('poly', PolynomialFeatures(degree=5)),
                 ('lasso', Lasso(max_iter=100000))])
for alpha in np.logspace(-5, 13, 19):
    pipe = pipe.set_params(lasso__alpha=alpha).fit(lsop_train, y_train)
    coefs[alpha] = pipe.named_steps['lasso'].coef_[1:]

coefs.T

In [ ]:
fig, ax = plt.subplots()
plot_curve(lasso, 'lasso__alpha', [1e-5, 1e-2, 1e-1, 1e1, 1e7], ax)

## Support Vector regression

Support vector machines can be used for regression purposes too. The main idea is to:
a) reduce the number of required training points to the support vectors
b) fit a linear model
c) transform data points into higher dimensions and fit the linear model in that higher space then transform the fitted curve to the original, lower dimension
d) instead of actually transforming the data, use kernel functions

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr = SVR(kernel='rbf', C=1e3, gamma=5e-5, degree=2)
svr.fit(lsop_train, y_train)
show_score(svr, lsop, y)

In [ ]:
y_hat = svr.predict(lsop_test)
plot_pred(y_test, y_hat)

In [ ]:
fig, ax = plt.subplots()
plot_curve(svr, 'kernel', ['linear', 'poly', 'rbf'], ax)

## [XGBoost](https://xgboost.readthedocs.io/en/latest/model.html)

XGBoost is short for **Extreme Gradient Boosting** which is a Gradient Boosted Tree method. Boosted tree is an **ensemble method**, basically training multiple trees on the same training set results a more robust solution. It is important that boosted trees incorporates a **regularization term** in its objective function. In this sense, boosted trees are the same as random forests. The difference comes from the training process. 

XGBoost use additive training: in each step it adds individual trees by selecting the best tree each time. The best tree is the **simplest tree** (tree structure score is minimal) **with the most information gain**.

For more detailed explanation please consult with these [slides](http://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf) and this [tutorial](https://xgboost.readthedocs.io/en/latest/model.html) or with this [wiki page](https://en.wikipedia.org/wiki/Gradient_boosting) on gradient boosting. To **install on windows**, please follow the instructions found [here](http://www.picnet.com.au/blogs/guido/post/2016/09/22/xgboost-windows-x64-binaries-for-download/).

In [ ]:
from xgboost.sklearn import XGBRegressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(lsop_train, y_train)
y_hat = xgb.predict(lsop_test)
show_score(xgb, lsop, y)

In [ ]:
plot_pred(y_test, y_hat)

In [ ]:
xgb

In [ ]:
fig, ax = plt.subplots()
plot_curve(xgb, 'n_estimators', [1, 5, 10, 25, 100], ax)

---

## Embedding pipelines

Trained pipelines can be used outside of the training program as well.

### Saving pipelines

First, we have to `serialize` the models. This process will save the whole pipeline object into a file. After saving, we can freely move the file and read it in elsewhere.  
**Important** to know that the used libraries must be the same versions in the saving and the loading end.

In [ ]:
import pickle

with open('xgboost_model.pickle', 'wb') as picklefile:
    pickle.dump(reg, picklefile)

### Loading pipelines

Loading and using the models is pretty easy - as long as we have the same libraries installed (and the same versions).

In [ ]:
import pickle

with open('xgboost_model.pickle', 'rb') as picklefile:
    model = pickle.load(picklefile)

In [ ]:
show_score(model, X, y)

## Kaggle challange

What to do:

- download dataset
- determine problem type
- examine features (description, insight)
- brainstorm possible features
- construct pipelines
- evaluate